In [16]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import torch
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers.tokenization_utils_base")



In [17]:
# Load CLIP model and processor
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def get_features(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = clip_processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = clip_model.get_image_features(**inputs)
    return outputs.squeeze().numpy()

def load_data_from_folder(folder):
    images = []
    labels = []
    for label, gender in enumerate(["FEMALE", "MALE"]):
        folder_path = os.path.join(folder, gender)
        if not os.path.exists(folder_path):
            print(f"Directory {folder_path} does not exist.")
            continue
        for filename in os.listdir(folder_path):
            if filename.endswith(".bmp"):
                image_path = os.path.join(folder_path, filename)
                features = get_features(image_path)
                images.append(features)
                labels.append(label)
    return np.array(images), np.array(labels)

In [18]:
# Update the path with the correct syntax
data_folder = r"C:\Random chetta\archive\CCTV Gender Classifier Dataset"

X, y = load_data_from_folder(data_folder)

from sklearn.model_selection import train_test_split

# Split Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the TensorFlow model with dropout for regularization
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X.shape[1],)),  # CLIP feature size
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),  # Adding dropout for regularization
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [26]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

def predict_gender(image_path):
    features = get_features(image_path)
    features = np.expand_dims(features, axis=0)  # Add batch dimension
    prediction = model.predict(features)
    return 'Male' if prediction > 0.5 else 'Female'

# Test the prediction function
test_image_path = r"man.jpeg"  # Replace with an actual test image path
print(f"The predicted gender is: {predict_gender(test_image_path)}")

Epoch 1/100
50/50 [==============================] - 1s 11ms/step - loss: 0.0298 - accuracy: 0.9899 - val_loss: 1.3908 - val_accuracy: 0.8283
Epoch 2/100
50/50 [==============================] - 0s 8ms/step - loss: 0.0193 - accuracy: 0.9912 - val_loss: 1.4359 - val_accuracy: 0.8258
Epoch 3/100
50/50 [==============================] - 0s 8ms/step - loss: 0.0112 - accuracy: 0.9968 - val_loss: 1.7570 - val_accuracy: 0.8308
Epoch 4/100
50/50 [==============================] - 0s 8ms/step - loss: 0.0235 - accuracy: 0.9905 - val_loss: 1.5650 - val_accuracy: 0.8333
Epoch 5/100
50/50 [==============================] - 0s 7ms/step - loss: 0.0233 - accuracy: 0.9918 - val_loss: 1.6762 - val_accuracy: 0.8359
Epoch 6/100
50/50 [==============================] - 0s 8ms/step - loss: 0.0220 - accuracy: 0.9912 - val_loss: 1.4472 - val_accuracy: 0.8308
Epoch 7/100
50/50 [==============================] - 0s 7ms/step - loss: 0.0181 - accuracy: 0.9931 - val_loss: 1.4981 - val_accuracy: 0.8283
Epoch 8/100


Epoch 1/10
50/50 [==============================] - 1s 11ms/step - loss: 0.4977 - accuracy: 0.7551 - val_loss: 0.4022 - val_accuracy: 0.8359
Epoch 2/10
50/50 [==============================] - 0s 6ms/step - loss: 0.3989 - accuracy: 0.8169 - val_loss: 0.3603 - val_accuracy: 0.8333
Epoch 3/10
50/50 [==============================] - 0s 5ms/step - loss: 0.3682 - accuracy: 0.8359 - val_loss: 0.3629 - val_accuracy: 0.8258
Epoch 4/10
50/50 [==============================] - 0s 6ms/step - loss: 0.3488 - accuracy: 0.8434 - val_loss: 0.3673 - val_accuracy: 0.8409
Epoch 5/10
50/50 [==============================] - 0s 6ms/step - loss: 0.3318 - accuracy: 0.8485 - val_loss: 0.3679 - val_accuracy: 0.8283
Epoch 6/10
50/50 [==============================] - 0s 6ms/step - loss: 0.3240 - accuracy: 0.8542 - val_loss: 0.3445 - val_accuracy: 0.8384
Epoch 7/10
50/50 [==============================] - 0s 6ms/step - loss: 0.3085 - accuracy: 0.8643 - val_loss: 0.3423 - val_accuracy: 0.8409
Epoch 8/10
50/50 [=